<a href="https://colab.research.google.com/github/FNAGHAV/Alphabet-Soup-Funding-Prediction/blob/main/AlphabetSoupCharity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import dependencies

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# Import and read charity_data.csv

application_df = pd.read_csv('https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv')
application_df.head(5)

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Binning ASK_AMT column
application_df['ASK_AMT_Binned'] = pd.cut(application_df['ASK_AMT'], bins=1000, labels=False)

# Finding the range of values for bin 0
bin_0_values = application_df.loc[application_df['ASK_AMT_Binned'] == 17, 'ASK_AMT']
bin_0_min = bin_0_values.min()
bin_0_max = bin_0_values.max()

print("Range of values in bin 0:", bin_0_min, "---", bin_0_max)
print('\n\n', application_df['ASK_AMT_Binned'].value_counts()[application_df['ASK_AMT_Binned'].value_counts() >= 50])

Range of values in bin 0: 148183962 --- 150830465


 ASK_AMT_Binned
0    33857
1      138
2       54
Name: count, dtype: int64


In [ ]:
# Drop non-beneficial ID columns

application_df = application_df[application_df['ASK_AMT'] <= 15000000]
application_df.drop(columns=['EIN', 'NAME'], inplace = True)
application_df.head(1)

<ipython-input-4-116550322d04>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  application_df.drop(columns=['EIN', 'NAME'], inplace = True)


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,ASK_AMT_Binned
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1,0


In [ ]:
# Getting number of uniqeu values for each column

application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              70
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8425
IS_SUCCESSFUL                2
ASK_AMT_Binned               2
dtype: int64

In [ ]:
# Looking into APPLICATION_TYPE and each values for each category

application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3     26759
T4      1541
T6      1216
T5      1168
T19     1065
T8       737
T7       724
T10      528
T9       122
T13       66
T12       26
T2        16
T25        3
T29        2
T15        2
T14        1
T17        1
Name: count, dtype: int64

In [ ]:
# Iterates through columns

for column in application_df.columns:
    if application_df[column].nunique() > 10:
        print('Number of datapoints for each unique value in column', column, 'is', application_df[column].value_counts())

Number of datapoints for each unique value in column APPLICATION_TYPE is APPLICATION_TYPE
T3     26759
T4      1541
T6      1216
T5      1168
T19     1065
T8       737
T7       724
T10      528
T9       122
T13       66
T12       26
T2        16
T25        3
T29        2
T15        2
T14        1
T17        1
Name: count, dtype: int64
Number of datapoints for each unique value in column CLASSIFICATION is CLASSIFICATION
C1000    17092
C2000     6039
C1200     4807
C3000     1913
C2100     1880
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 70, dtype: int64
Number of datapoints for each unique value in column ASK_AMT is ASK_AMT
5000     25398
15583        3
10478        3
6725         3
63981        3
         ...  
51641        1
85869        1
12426        1
16406        1
20197        1
Name: count, Length: 8425, dtype: int64


In [ ]:
# Choose a cutoff value and create a list of application types to be replaced

cutoff_app = 500

app_type_count = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = app_type_count[app_type_count < cutoff_app].index.to_list()


# Replacing rare values with Other
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, 'Other')

application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       26759
T4        1541
T6        1216
T5        1168
T19       1065
T8         737
T7         724
T10        528
Other      239
Name: count, dtype: int64

In [ ]:
# Look into CLASSIFICATION column and its value counts

application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17092
C2000     6039
C1200     4807
C3000     1913
C2100     1880
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 70, dtype: int64

In [ ]:
# Look at value counts > 1

application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() > 1]

CLASSIFICATION
C1000    17092
C2000     6039
C1200     4807
C3000     1913
C2100     1880
C7000      774
C1700      282
C4000      194
C5000      116
C1270      113
C2700      104
C2800       94
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       31
C1240       30
C8000       19
C7120       18
C1500       15
C6000       15
C1800       14
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C0           3
C2710        3
C1234        2
C1267        2
C1246        2
C1256        2
C3200        2
Name: count, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced

cutoff_class = 1000

class_count = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = class_count[class_count < cutoff_class].index.to_list()


# Replacing rare values with Other

for classes in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(classes, 'Other')

application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17092
C2000     6039
C1200     4807
Other     2246
C3000     1913
C2100     1880
Name: count, dtype: int64

In [ ]:
# Convert categorical data to numeric

application_df = pd.get_dummies(application_df, dtype=int)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,ASK_AMT_Binned,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0


In [ ]:
# Splitting data into features (X) and target (y)

X = application_df.drop(columns=['IS_SUCCESSFUL'])
y = application_df['IS_SUCCESSFUL']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Create standard scaler instane, fitting it, and scaling it

scaler = StandardScaler()

X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model and compile

nn = tf.keras.models.Sequential()

nn.add(tf.keras.layers.Dense(units = 30, activation = 'relu', input_shape = (X_train_scaled.shape[1],)))
nn.add(tf.keras.layers.Dense(units = 15, activation = 'relu'))
nn.add(tf.keras.layers.Dense(units = 15, activation = 'relu'))
nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                1350      
                                                                 
 dense_1 (Dense)             (None, 15)                465       
                                                                 
 dense_2 (Dense)             (None, 15)                240       
                                                                 
 dense_3 (Dense)             (None, 1)                 16        
                                                                 
Total params: 2071 (8.09 KB)
Trainable params: 2071 (8.09 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Creating callback instances to be called

callbacks = ModelCheckpoint(filepath = 'model_weights_epochs_{epoch:02d}.weights.h5', save_weights_only = True,
                           save_freq = 5)
# Train model

fit_model = nn.fit(X_train_scaled, y_train, epochs= 100, validation_split = 0.25, verbose = 1, callbacks = callbacks)

loss, accuracy = nn.evaluate(X_test_scaled, y_test, verbose = 2)

print('Accuracy is:', accuracy, '\nLoss is:', loss)

nn.save('AlphabetSoupCharity.h5')

Epoch 1/100
558/558 [==============================] - 8s 11ms/step - loss: 0.5825 - accuracy: 0.7100 - val_loss: 0.5623 - val_accuracy: 0.7262
Epoch 2/100
558/558 [==============================] - 5s 8ms/step - loss: 0.5575 - accuracy: 0.7260 - val_loss: 0.5569 - val_accuracy: 0.7289
Epoch 3/100
558/558 [==============================] - 6s 10ms/step - loss: 0.5539 - accuracy: 0.7281 - val_loss: 0.5556 - val_accuracy: 0.7296
Epoch 4/100
558/558 [==============================] - 8s 14ms/step - loss: 0.5518 - accuracy: 0.7286 - val_loss: 0.5551 - val_accuracy: 0.7311
Epoch 5/100
558/558 [==============================] - 5s 9ms/step - loss: 0.5501 - accuracy: 0.7301 - val_loss: 0.5550 - val_accuracy: 0.7302
Epoch 6/100
558/558 [==============================] - 6s 11ms/step - loss: 0.5490 - accuracy: 0.7317 - val_loss: 0.5535 - val_accuracy: 0.7292
Epoch 7/100
558/558 [==============================] - 5s 9ms/step - loss: 0.5483 - accuracy: 0.7311 - val_loss: 0.5522 - val_accuracy: 0.

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
